<a href="https://colab.research.google.com/github/Sparlly007/AncestryDesignProblem/blob/master/NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install scikit-learn
!pip install scikit-learn

from typing import Callable, List, Tuple
from datasets import load_dataset
import datasets
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
import sklearn
from sklearn.metrics import accuracy_score
import collections
import re
nltk.download('punkt')



In [20]:
precision = datasets.load_metric("precision")
dataSet = load_dataset('imdb', 'plain_text', split='train')
testDataSet = load_dataset('imdb', 'plain_text', split='test')

smsDataSet = load_dataset('sms_spam', 'plain_text', split='train')
smsDataSet[:15]['sms'][13]

Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
Reusing dataset sms_spam (/root/.cache/huggingface/datasets/sms_spam/plain_text/1.0.0/53f051d3b5f62d99d61792c91acefe4f1577ad3e4c216fb0ad39e30b9f20019c)


"I've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times.\n"

In [ ]:
dataSet[:3]['text'][2]
#reference_batch = smsDataSet[:3]['sms']
#sys_batch = smsDataSet[:15]['sms'][13]
#precision.add_batch(predictions=sys_batch, references=reference_batch)
#score = precision.compute()
#score
#dataSet.features['label']

In [22]:
def bag_of_words(cleaned : List[str]):
  # create a set containing all the tokens that are only letters
  # the set will automatically filter out duplicates
  words = {w.lower() for w in cleaned if w.isalpha()}
  # convert the set to a list before returning.
  return list(words)

In [23]:
def bag_of_words(cleaned : List[str]):
  # create a set containing all the tokens that are only letters
  # the set will automatically filter out duplicates
  words = {w.lower() for w in cleaned if w.isalpha()}
  # convert the set to a list before returning.
  return list(words)

In [24]:
def clean(text: str):
  test = ""
  for i in text:
    test = test+i
  test = test.lower()
  #Removes punctuation
  res = re.sub(r'[^\w\s]', '', test)
  #Removes numbers
  res = ''.join([i for i in res if not i.isdigit()])
  #Tokenizes string by space
  #res = word_tokenize(res)
  #Removes one-char words
  #res = [i for i in res if len(i) > 1]
  return res

#clean(dataSet[:20]['text'])

In [25]:
def learn(data: List[Tuple[str, int]]):
  vocab = [] #Our list of vocabs included in the dataset
  docs = [] #Will contain the list of reviews of the dataset after cleaning them
  grouped = {} #Contains classes with their documents combined
  classFreq = {} #Displays frequency of words of each class
  probTable = {} #Will contain the probability of each feature in context of its class
  probClass = {} #Probability of each class among total documents
  features = {} #Will contain features to be extracted

  #Combines document of the same classes to their respective classes through dictionary
  for i, j in data:
    docs.append(i)
    i = clean(i)
    if j not in grouped.keys():
      grouped[j] = i
      probClass[j] = 1
    else:
      grouped[j] += i
      probClass[j] += 1

  #Creates vocab list  
  for i, j in data:
    for k in word_tokenize(i):
      if k not in vocab:
        vocab.append(k)

  #Extracts features with the bag-of-words feature extractor
  for i in grouped.keys():
    features[i] = bag_of_words(grouped[i])
    
  #Gets probability for every class to appear among all docs
  for i in probClass.keys():
    probClass[i] = probClass[i] / len(docs)

  #Attaches a counter to each dictionary entry
  for i, j in data:
    classFreq[j] = collections.Counter(grouped[j])

  #Attaches a dictionary of feature probabilities to each class
  for i in classFreq.keys():
    probTable[i] = {}
    for j in classFreq[i]:
      if j in features[i]:
        prob = (classFreq[i][j] + 1) / (len(classFreq[i]) + 1)
        probTable[i][j] = prob

  return probTable, probClass, vocab


In [ ]:
def classify(document: str, model, classProb, vocab):
  sum = {}
  document = clean(document)
  for i in classProb.keys():
    sum[i] = classProb[i]
  
  for i in model.keys():
    for j in document:
      if j in vocab and j in model[i].keys():
        sum[i] = sum[i] * model[i][j]

  return max(sum, key=sum.get)


In [27]:
#vocab = [] #Our list of vocabs included in the dataset
docs = [] #Will contain the list of reviews of the dataset after cleaning them
#text = dataSet[:3]['text'] #List of all reviews in dataset
#classes = dataSet[:3]['label'] #List of all classes in dataset to each review respective of index
text = smsDataSet[:10]['sms'] #List of all reviews in dataset
classes = smsDataSet[:10]['label'] #List of all classes in dataset to each review respective of index
grouped = {}
classFreq = {} #Displays frequency of words of each class
probTable = {}

#Concatenates each review to a list
for i in range(len(text)):
   docs.append(text[i])

rest = word_tokenize(dataSet[:3]['text'][0])
#for x in tokenized: 
#  if x not in vocab:
#    vocab.append(x)

#Classifies each string with its class
classified = []

for i in range(len(docs)):
  classified.append((docs[i], classes[i]))

result = learn(classified)
result
classify(smsDataSet[:15]['sms'][10], result[0], result[1], result[2])
#bag_of_words(rest)

#grouped = {}
#for i, j in classified:
 #   i = clean(i)
 #   if j not in grouped.keys():
 #     grouped[j] = i
#    else:
 #     grouped[j] += i

#bag_of_words(grouped[0])
#grouped[1]

0